# Introduction

This notebook compares the original NN and the PSO-NN for different datasets.

The dataset can be used via libraries like `sklearn` or `tensorflow`. The datasets used are:
- Iris
- Wine
- Breast Cancer

Comparing the performance of PSO-based optimization with traditional backpropagation
in terms of accuracy and convergence speed.

✓ Comparing behaviour and results in three different datasets your choice. You are
allowed to choose any, but you are expected to reason, explain and compare the results.

✓ Analyzing the influence of various PSO parameters (w, c1, c2, and velocity limits) on
optimization performance.

✓ Identifying strengths and weaknesses of PSO for neural network training, particularly in
terms of handling high-dimensional search spaces.

In [4]:
#Loading all 3 dataset and creating train test splits.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#import iris dataset from sklearn
from sklearn.datasets import load_iris,load_breast_cancer,load_wine
from sklearn.preprocessing import LabelEncoder


#Loading the dataset from the internet
datasets = [load_iris(),load_breast_cancer(),load_wine()]
dataset_names = ['iris','breast_cancer','wine']
#create train test splits for each dataset and put them in a dictionary
sets = []
for i,dataset in enumerate(datasets):
    datadict ={}
    # Do the split
    # Please don't change the test_train_ration
    data = dataset
    TEST_SPLIT_RATIO = 0.2
    X = data['data']
    y = data['target']
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT_RATIO, random_state=42)
    
    datadict['X_train'] = X_train
    datadict['X_test'] = X_test
    datadict['y_train'] = y_train
    datadict['y_test'] = y_test
    sets.append(datadict)
    
print(sets[0].keys())


dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


# Performance Comparison

Comparing the performance of PSO-based optimization with traditional backpropagation
in terms of accuracy and convergence speed.

In [15]:
import importlib
import timeit
import seaborn as sns
import matplotlib.pyplot as plt
# Import modules
PSO = importlib.import_module("PSO-NN")
CS = importlib.import_module("classic-NN")
from commonsetup import n_hidden, X_train, X_test, y_train, y_test, n_inputs, n_classes, activation, n_iteration, learning_rate

# Set PSO parameters
par_C1 = 2.0
par_C2 = 2.0
par_W = 0.7
par_SwarmSize = 100
batchsize = 50  # Number of data instances used by the fitness function
#n_iteration = 1000000

# Initialize results storage
results = []

#setss = [sets[1]]

# Iterate over datasets
for i, datadict in enumerate(sets):
    
    print(f"\n#### Testing on {dataset_names[i]} dataset ####\n")

    # Extract train/test data
    X_train = datadict['X_train']
    X_test = datadict['X_test']
    y_train = datadict['y_train']
    y_test = datadict['y_test']
    
    # Dynamically set inputs and classes
    n_inputs = X_train.shape[1]
    n_classes = len(np.unique(y_train))

    # Print settings
    print("Number of hidden layers:", n_hidden)
    print("Activation function:", activation[0])
    print("Number of variables to optimize:", (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes)
    print("PSO parameters C1:", par_C1, "C2:", par_C2, "W:", par_W, "SwarmSize:", par_SwarmSize, "Iterations:", n_iteration)
    n_iteration = 50
    accs = [0]
    traintimes = [0]
    stepsize = 5
    for n_i in range(n_iteration)[::stepsize]:
        # PSO Optimization
        nn_pso = PSO.NeuralNetwork(n_inputs, n_hidden, n_classes, activation[0])
        def optimize_pso():
            pso = PSO.PSOOptimizer(nn_pso, par_C1, par_C2, par_W, par_SwarmSize, n_i, batchsize)
            weights = pso.optimize(X_train, y_train,verbose=False)
            return weights

        pso_time = timeit.timeit(optimize_pso, number=1)
        traintimes.append(pso_time)
        weights = optimize_pso()
        y_pred_pso = nn_pso.predict(weights, X_test)
        pso_accuracy = (y_pred_pso == y_test).mean()
        accs.append(pso_accuracy)

        print(f"PSO optimization time: {pso_time:.2f} seconds")
        print(f"Accuracy PSO-NN: {pso_accuracy:.2f}")
    
    print(len(accs))
    print(len(traintimes))
    print(len(range(n_iteration)[::stepsize]))
    #plott acuracy and training times to n iterations using seaborn
    sns.lineplot(x=range(n_iteration)[::stepsize],y=accs[1:],label='Accuracy')
    sns.lineplot(x=range(n_iteration)[::stepsize],y=traintimes[1:],label='Training Time')
    plt.show()

    # Backpropagation Training
    def train_backprop():
        nn_classic = CS.NeuralNetwork(n_inputs, n_hidden, n_classes, activation[0], activation[1])
        nn_classic.train(X_train, y_train, n_iteration, learning_rate, verbose=False)
        return nn_classic

    backprop_time = timeit.timeit(train_backprop, number=1)
    nn_classic = train_backprop()
    y_pred_classic = nn_classic.predict(X_test)
    backprop_accuracy = (y_pred_classic == y_test).mean()

    print(f"Backpropagation training time: {backprop_time:.2f} seconds")
    print(f"Accuracy Classic-NN: {backprop_accuracy:.2f}")

    # Store results
    results.append({
        'dataset': dataset_names[i],
        'pso_accuracy': pso_accuracy,
        'pso_time': pso_time,
        'backprop_accuracy': backprop_accuracy,
        'backprop_time': backprop_time
    })

# Print overall comparison
print("\n#### Overall Comparison Results ####\n")
for result in results:
    print(f"Dataset: {result['dataset']}")
    print(f"PSO-NN Accuracy: {result['pso_accuracy']:.2f} | Time: {result['pso_time']:.2f} seconds")
    print(f"Classic-NN Accuracy: {result['backprop_accuracy']:.2f} | Time: {result['backprop_time']:.2f} seconds\n")


#### Testing on iris dataset ####

Number of hidden layers: 10
Activation function: <function sigmoid at 0x00000163E1E10940>
Number of variables to optimize: 83
PSO parameters C1: 2.0 C2: 2.0 W: 0.7 SwarmSize: 100 Iterations: 1000


AttributeError: 'float' object has no attribute 'copy'

Classical NN is way faster but has a worse accuracy than PSO-NN. PSO-NN is slower but has a better accuracy. This is still not perfect since classical NN is highly dependent on the initial weights and the learning rate. PSO-NN is more stable and has a better accuracy. A random seed is used to make the comparision reliable.